# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 15 2023 10:06AM,259361,21,849369,"NBTY Global, Inc.",Released
1,Mar 15 2023 9:51AM,259360,19,ADV80013740,"AdvaGen Pharma, Ltd",Released
2,Mar 15 2023 9:51AM,259360,19,ADV80013741,"AdvaGen Pharma, Ltd",Released
3,Mar 15 2023 9:51AM,259360,19,ADV80013769,"AdvaGen Pharma, Ltd",Released
4,Mar 15 2023 9:51AM,259360,19,ADV80013771,"AdvaGen Pharma, Ltd",Released
5,Mar 15 2023 9:51AM,259360,19,ADV80013772,"AdvaGen Pharma, Ltd",Released
6,Mar 15 2023 9:51AM,259360,19,ADV80013785,"AdvaGen Pharma, Ltd",Released
7,Mar 15 2023 9:51AM,259360,19,ADV80013763,"AdvaGen Pharma, Ltd",Released
8,Mar 15 2023 9:51AM,259360,19,ADV80013764,"AdvaGen Pharma, Ltd",Released
9,Mar 15 2023 9:51AM,259360,19,ADV80013765,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,259357,Executing,2
17,259357,Released,7
18,259358,Released,1
19,259360,Released,24
20,259361,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259354,NaN,1.0
259357,2.0,7.0
259358,NaN,1.0
259360,NaN,24.0
259361,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259190,3.0,40.0
259255,5.0,16.0
259258,0.0,1.0
259291,0.0,6.0
259311,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259190,3,40
259255,5,16
259258,0,1
259291,0,6
259311,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259190,3,40
1,259255,5,16
2,259258,0,1
3,259291,0,6
4,259311,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259190,3,40
1,259255,5,16
2,259258,,1
3,259291,,6
4,259311,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 15 2023 10:06AM,259361,21,"NBTY Global, Inc."
1,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd"
25,Mar 15 2023 9:39AM,259255,15,"Brookfield Pharmaceuticals, LLC"
46,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc."
52,Mar 15 2023 9:36AM,259325,15,"Person & Covey, Inc."
57,Mar 15 2023 9:35AM,259190,15,"Brookfield Pharmaceuticals, LLC"
100,Mar 15 2023 9:29AM,259358,10,"iOpen World, LLC"
101,Mar 15 2023 9:27AM,259357,10,Bio-PRF
110,Mar 15 2023 9:14AM,259354,20,"Granules Pharmaceuticals, Inc."
111,Mar 15 2023 9:10AM,259353,10,"Digital Brands, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 15 2023 10:06AM,259361,21,"NBTY Global, Inc.",,1
1,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd",,24
2,Mar 15 2023 9:39AM,259255,15,"Brookfield Pharmaceuticals, LLC",5,16
3,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc.",,6
4,Mar 15 2023 9:36AM,259325,15,"Person & Covey, Inc.",,5
5,Mar 15 2023 9:35AM,259190,15,"Brookfield Pharmaceuticals, LLC",3,40
6,Mar 15 2023 9:29AM,259358,10,"iOpen World, LLC",,1
7,Mar 15 2023 9:27AM,259357,10,Bio-PRF,2,7
8,Mar 15 2023 9:14AM,259354,20,"Granules Pharmaceuticals, Inc.",,1
9,Mar 15 2023 9:10AM,259353,10,"Digital Brands, LLC",,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 15 2023 10:06AM,259361,21,"NBTY Global, Inc.",1,
1,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd",24,
2,Mar 15 2023 9:39AM,259255,15,"Brookfield Pharmaceuticals, LLC",16,5
3,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc.",6,
4,Mar 15 2023 9:36AM,259325,15,"Person & Covey, Inc.",5,
5,Mar 15 2023 9:35AM,259190,15,"Brookfield Pharmaceuticals, LLC",40,3
6,Mar 15 2023 9:29AM,259358,10,"iOpen World, LLC",1,
7,Mar 15 2023 9:27AM,259357,10,Bio-PRF,7,2
8,Mar 15 2023 9:14AM,259354,20,"Granules Pharmaceuticals, Inc.",1,
9,Mar 15 2023 9:10AM,259353,10,"Digital Brands, LLC",14,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 15 2023 10:06AM,259361,21,"NBTY Global, Inc.",1,
1,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd",24,
2,Mar 15 2023 9:39AM,259255,15,"Brookfield Pharmaceuticals, LLC",16,5
3,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc.",6,
4,Mar 15 2023 9:36AM,259325,15,"Person & Covey, Inc.",5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 15 2023 10:06AM,259361,21,"NBTY Global, Inc.",1.0,NaN
1,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd",24.0,NaN
2,Mar 15 2023 9:39AM,259255,15,"Brookfield Pharmaceuticals, LLC",16.0,5.0
3,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc.",6.0,NaN
4,Mar 15 2023 9:36AM,259325,15,"Person & Covey, Inc.",5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 15 2023 10:06AM,259361,21,"NBTY Global, Inc.",1.0,0.0
1,Mar 15 2023 9:51AM,259360,19,"AdvaGen Pharma, Ltd",24.0,0.0
2,Mar 15 2023 9:39AM,259255,15,"Brookfield Pharmaceuticals, LLC",16.0,5.0
3,Mar 15 2023 9:36AM,259291,15,"Alliance Pharma, Inc.",6.0,0.0
4,Mar 15 2023 9:36AM,259325,15,"Person & Covey, Inc.",5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1556084,40.0,3.0
12,259312,0.0,1.0
15,1037061,67.0,8.0
19,1296622,26.0,5.0
20,259354,1.0,0.0
21,259361,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1556084,40.0,3.0
1,12,259312,0.0,1.0
2,15,1037061,67.0,8.0
3,19,1296622,26.0,5.0
4,20,259354,1.0,0.0
5,21,259361,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,40.0,3.0
1,12,0.0,1.0
2,15,67.0,8.0
3,19,26.0,5.0
4,20,1.0,0.0
5,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,12,Released,0.0
2,15,Released,67.0
3,19,Released,26.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,3.0,1.0,8.0,5.0,0.0,0.0
Released,40.0,0.0,67.0,26.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,3.0,1.0,8.0,5.0,0.0,0.0
1,Released,40.0,0.0,67.0,26.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,3.0,1.0,8.0,5.0,0.0,0.0
1,Released,40.0,0.0,67.0,26.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()